In [13]:
import ccxt
import pandas as pd
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
from binance.client import Client
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
import talib as ta
import ta
import sqlite3

In [14]:
load_dotenv()  # Esto carga las variables de entorno desde `.env`

api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

In [15]:

# Conexión a la base de datos con los datos sacados en tiempo real
conn_realtime = sqlite3.connect(r'C:\Users\lenovo\Desktop\Conexion Api Binance\crypto_data.db')

# Consulta SQL para obtener los datos en tiempo real
# Ajusta el nombre de la tabla según corresponda
query_realtime = "SELECT * FROM btcusdt_1h_data"

# Ejecutar la consulta y guardar los resultados en un DataFrame
df_realtime = pd.read_sql_query(query_realtime, conn_realtime)

# Cerrar la conexión a la base de datos en tiempo real
conn_realtime.close()

# Cliente de Binance
client = Client(api_key, api_secret)

def fetch_binance_data(symbol, interval, limit=1000):
    bars = client.get_historical_klines(symbol, interval, "1 Jan, 2017", limit=limit)
    df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']]
    return df

symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT']
intervals = ['1h']  # Solo intervalos horarios

data = {}  # Diccionario para almacenar DataFrames
raw_data_dir = "../data/raw"
os.makedirs(raw_data_dir, exist_ok=True)

for symbol in symbols:
    data[symbol] = {}
    for interval in intervals:
        try:
            print(f"Fetching data for {symbol} at interval {interval}")
            df = fetch_binance_data(symbol, interval)
            data[symbol][interval] = df
            # Guardar los datos descargados
            df.to_csv(f"{raw_data_dir}/{symbol}_{interval}.csv")
            print(f"Data fetched and saved for {symbol} at interval {interval}. Data shape: {df.shape}")
            time.sleep(60)  # Delay para no violar los límites de la API
        except Exception as e:
            print(f"Error fetching data for {symbol} at interval {interval}: {str(e)}")

# Concatenar los datos históricos con los de tiempo real y renombrar el dataframe
df_concatenated = pd.concat([df_realtime, df])

Fetching data for BTCUSDT at interval 1h
Data fetched and saved for BTCUSDT at interval 1h. Data shape: (59040, 5)
Fetching data for ETHUSDT at interval 1h
Data fetched and saved for ETHUSDT at interval 1h. Data shape: (59040, 5)
Fetching data for XRPUSDT at interval 1h
Data fetched and saved for XRPUSDT at interval 1h. Data shape: (52836, 5)
Fetching data for LTCUSDT at interval 1h
Data fetched and saved for LTCUSDT at interval 1h. Data shape: (56215, 5)


In [25]:

# Con los dataframes concatenados pre procesamos los datos y guardamos los datos en un nuevo base de datos sql (crypto_data_proc.db)
class CryptoDataProcessor:
    def __init__(self, df_concatenated, conn, table_name, symbol):
        self.df = df_concatenated
        self.conn = conn
        self.table_name = table_name
        self.symbol = symbol
        # Inicialización basada en análisis de puntos extremos iniciales
        self.initialize_support_resistance()
        self.breakout_threshold = 0.01  # 1% por encima o por debajo

    def initialize_support_resistance(self):
        # Establecer soporte inicial como el mínimo de los primeros días
        self.support = min(self.df[f'{self.symbol}_low'].iloc[:30])
        # Establecer resistencia inicial como el máximo de los primeros días
        self.resistance = max(self.df[f'{self.symbol}_high'].iloc[:30])
        self.df[f'{self.symbol}_support'] = self.support
        self.df[f'{self.symbol}_resistance'] = self.resistance

    def update_support_resistance(self):
        # Crear columnas para soporte y resistencia dinámicos
        self.df[f'{self.symbol}_dynamic_support'] = self.support
        self.df[f'{self.symbol}_dynamic_resistance'] = self.resistance

        for index, row in self.df.iterrows():
            current_low = row[f'{self.symbol}_low']
            current_high = row[f'{self.symbol}_high']
            current_close = row[f'{self.symbol}_close']

            # Revisar y ajustar soporte y resistencia
            if current_close > self.resistance:
                self.support = self.resistance  # La resistencia rota se convierte en soporte
                self.resistance = current_high
            elif current_close < self.support:
                self.resistance = self.support  # El soporte roto se convierte en resistencia
                self.support = current_low

            # Actualizar el DataFrame con los valores dinámicos
            self.df.at[index, f'{self.symbol}_dynamic_support'] = self.support
            self.df.at[index, f'{self.symbol}_dynamic_resistance'] = self.resistance         

    def add_technical_indicators(self):
        # Calcular SMAs y EMAs con manejo inicial dinámico para los datos iniciales limitados
        windows = [7, 14, 21, 28, 50, 100, 200]
        for i in windows:
            self.df[f'{self.symbol}_sma_{i}'] = self.df[f'{self.symbol}_close'].rolling(window=i, min_periods=1).mean()
            self.df[f'{self.symbol}_ema_{i}'] = self.df[f'{self.symbol}_close'].ewm(span=i, adjust=False, min_periods=1).mean()

        # RSI personalizado
        delta = self.df[f'{self.symbol}_close'].diff()
        gain = delta.where(delta > 0, 0.0)
        loss = -delta.where(delta < 0, 0.0)

        # Usar mean() con min_periods=1 asegura que calculamos el valor incluso si hay menos datos que la ventana
        avg_gain = gain.rolling(window=14, min_periods=1).mean()
        avg_loss = loss.rolling(window=14, min_periods=1).mean()

        rs = avg_gain / avg_loss
        self.df[f'{self.symbol}_rsi_14'] = 100.0 - (100.0 / (1.0 + rs))

        # MACD
        macd = ta.trend.MACD(self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_macd'] = macd.macd()
        self.df[f'{self.symbol}_macd_signal'] = macd.macd_signal()
        self.df[f'{self.symbol}_macd_diff'] = macd.macd_diff()

        # Williams %R
        self.df[f'{self.symbol}_willr'] = ta.momentum.williams_r(
            self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'], lbp=14)

        # ATR
        self.df[f'{self.symbol}_atr_14'] = ta.volatility.average_true_range(
            self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'], window=14)

        # ADX
        self.df[f'{self.symbol}_adx'] = ta.trend.adx(
            self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'], window=14)

        # Commodity Channel Index
        self.df[f'{self.symbol}_cci'] = ta.trend.cci(
            self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'], window=20)
 
 def add_candle_patterns(self):
    # Patrones básicos y comunes
        self.df[f'{self.symbol}_doji'] = talib.CDLDOJI(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_engulfing'] = talib.CDLENGULFING(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_hammer'] = talib.CDLHAMMER(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_inverted_hammer'] = talib.CDLINVERTEDHAMMER(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_hanging_man'] = talib.CDLHANGINGMAN(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_shooting_star'] = talib.CDLSHOOTINGSTAR(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_morning_star'] = talib.CDLMORNINGSTAR(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_evening_star'] = talib.CDLEVENINGSTAR(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_morning_doji_star'] = talib.CDLMORNINGDOJISTAR(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_evening_doji_star'] = talib.CDLEVENINGDOJISTAR(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])

    # Patrones complejos
        self.df[f'{self.symbol}_piercing_line'] = talib.CDLPIERCING(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_dark_cloud_cover'] = talib.CDLDARKCLOUDCOVER(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_three_white_soldiers'] = talib.CDL3WHITESOLDIERS(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_three_black_crows'] = talib.CDL3BLACKCROWS(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_three_inside_up_down'] = talib.CDL3INSIDE(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_three_outside_up_down'] = talib.CDL3OUTSIDE(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_three_stars_in_the_south'] = talib.CDL3STARSINSOUTH(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])
        self.df[f'{self.symbol}_three_advancing_white_soldiers'] = talib.CDL3WHITESOLDIERS(
        self.df[f'{self.symbol}_open'], self.df[f'{self.symbol}_high'], self.df[f'{self.symbol}_low'], self.df[f'{self.symbol}_close'])

def process(self):
        self.initialize_support_resistance()
        self.add_technical_indicators()
        self.add_candle_patterns()
        self.update_support_resistance()
        # Añadir el precio de cierre de la siguiente hora como variable a predecir
        self.df[f'{self.symbol}_next_close'] = self.df[f'{self.symbol}_close'].shift(-1)
        self.df.to_sql(self.table_name, self.conn, if_exists='replace', index=False)

# Código para cargar y renombrar los DataFrames correctamente
data = {}
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT']
for symbol in symbols:
    file_path = f"{raw_data_dir}/{symbol}_1h.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
        # Añadir el prefijo del símbolo a las columnas
        df.columns = [f"{symbol}_{col}" for col in df.columns]
        data[symbol] = df
    else:
        print(f"No data file found for {symbol}")

processed_data = {symbol: CryptoDataProcessor(data[symbol], conn_crypto_data_proc, symbol.lower() + '_processed', symbol).process() 
                  for symbol in data}



IndentationError: unindent does not match any outer indentation level (<string>, line 83)

In [22]:
# Conectar a la base de datos donde se guardarán los datos procesados
conn_crypto_data_proc = sqlite3.connect('C:/Users/lenovo/Desktop/Conexion Api Binance/crypto_data_proc.db')

# Nombre de la tabla donde se guardarán los datos procesados
table_name = 'crypto_data_processed'

# Guardar los datos procesados en un nuevo DataFrame
processed_data = {}

# Después de procesar los datos, iterar sobre los datos para guardarlos en la base de datos
for symbol, df in processed_data.items():
    # Guardar el DataFrame en la base de datos
    df.to_sql(table_name, conn_crypto_data_proc, if_exists='replace', index=False)

In [23]:
# Conectar a la base de datos
conn = sqlite3.connect('C:/Users/lenovo/Desktop/Conexion Api Binance/crypto_data_proc.db')

# Realizar una consulta SQL para seleccionar todos los datos de la tabla
query = "SELECT * FROM crypto_data_processed"

# Leer los datos de la tabla en un DataFrame
df_from_db = pd.read_sql_query(query, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Imprimir el DataFrame
print(df_from_db)

DatabaseError: Execution failed on sql 'SELECT * FROM crypto_data_processed': no such table: crypto_data_processed